In [1]:
import csv
import re
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [2]:
## file name and setup 

file_name = 'index'

#pollutants
terms_of_interest = ['carcinogenic', 'carcinogens', 'arsenic', 'nitrate', 'nitrite', 'nitrates']

#urls
basic_url = 'https://www.communitywatercenter.org/news?n=448&page='
n_pages = 34

#cities
location_names = pd.read_csv('CA_names.csv')

In [4]:
pagelinks =[]

for number in range(1,n_pages+1):

    url = basic_url + str(number)
    page = urlopen(url)
    soup = BeautifulSoup(page, 'html.parser') #containing the HTML of the page
    name_box = soup.find_all('h3', attrs={'class': 'news_heading'})
    for count in range(0, len(name_box)):
            for link in name_box[count].find_all('a', href=True):
                pagelinks.append(link['href']) 

In [5]:
# For finding cities and counties

cities = location_names['city'].tolist() 
counties = location_names['county_name'].tolist()

## hack solution to control for weird city names matching words

cities.remove('August')
cities.remove('Challenge')
locations = cities + counties

#creating regex expressions
re_cities = '' + cities[0]
for i in cities[1:]:
    re_cities = re_cities + "|" + i
re_counties = '' + counties[0]
for i in counties[1:]:
    re_counties = re_counties + "|" + i
    
def intersect(a, b):
     return list(set(a) & set(b))

In [6]:
# Writing to CSV file (basic)

data = []

for count in range(0, len(pagelinks)):
    
    page_url = "https://www.communitywatercenter.org" + pagelinks[count]
    page = urlopen(page_url)
    soup = BeautifulSoup(page, 'html.parser')
    main = soup.find("div", id="content").text.strip()
    
    title = soup.title.string.replace('\r', '').replace('\n', '')
    date_section = soup.find('div', id="headline")
    date = date_section.find_all(string=re.compile("Published on"))
    city = list(set(re.findall(re_cities, main)))
    county = list(set(re.findall(re_counties, main)))
    
    for term in range(0, len(terms_of_interest)):

            phrase = [sentence + '.' for sentence in main.split('.') if terms_of_interest[term] in sentence]

            for item in range(0, len(phrase)):

                data.append([phrase[item], str(date)[19:-8], city, county, title, page_url])
phrase_df = pd.DataFrame(data, columns = ["phrase", "date", "city", "county", "title", "page_url"])
phrase_df.to_csv("phrase.csv")
phrase_df.head()                
#     with open(file_name + '.csv', 'a') as mycsv: ## a for append
        
#         writer = csv.writer(mycsv)
        
#         for term in range(0, len(terms_of_interest)):

#             phrase = [sentence + '.' for sentence in main.split('.') if terms_of_interest[term] in sentence]

#             for item in range(0, len(phrase)):

#                 writer.writerow([phrase[item], str(date)[19:-8], city, county, title, page_url])

,phrase,date,city,county,title,page_url
0,"\n\nNEW OFFICE IN WATSONVILLE, CA\n\nWe are e...","September 17, 2018","[Porterville, Watsonville, Walker, Visalia, Fr...","[Fresno, San Joaquin, Sacramento]",August 2018 eNews - Community Water Center,https://www.communitywatercenter.org/august201...
1,"Bartolo Chavez of Arvin, CA has drinking water...","September 06, 2018",[Arvin],[],Justice Delayed - Community Water Center,https://www.communitywatercenter.org/justicede...
2,"Bartolo Chavez of Arvin, CA has drinking water...","September 06, 2018",[Arvin],[],Justice Delayed - Community Water Center,https://www.communitywatercenter.org/justicede...
3,"org #NoToxicTaps\n\n1,2,3 - TCP\n\n1,2,3-trich...","July 31, 2018","[Porterville, Tulare, Walker, Visalia, Orosi, ...","[Kings, Del Norte, Tulare, Kern, San Diego, Lo...",July 2018 eNews - Community Water Center,https://www.communitywatercenter.org/july_2018...
4,Over 300 mostly rural and economically disadv...,"October 24, 2017",[],[],Small Communities Can't Solve Toxic Water Prob...,https://www.communitywatercenter.org/toxic_wat...


In [8]:
#read csv
output = pd.read_csv('phrase.csv', header= 0)
# isolate phrases and  without location

In [9]:
## for data analysis %load_ext rpy2.ipython 